In [478]:
import numpy as np
import xml.sax
import sys, os
import csv, re, glob, math, pickle
import datetime
import Stemmer
import nltk
# nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import *

from heapq import heapify, heappush, heappop 

In [479]:
STOPWORDS = set(stopwords.words('english')) 
URL_STOP_WORDS = set(["http", "https", "www", "ftp", "com", "net", "org", "archives", "pdf", "html", "png", "txt"])
UNWANTED_LINES= set([ "redirect", "align", "realign", "valign", "nonalign", "malign", "unalign", "salign", "qalign", "halign", "font", "fontsiz", "fontcolor", "backgroundcolor", "background", "style", "center", "text"])
fields= ['t', 'i', 'c', 'b', 'r', 'l']

docId=1
current_indexNum= 1
final_indexNum= 1
max_IndexSize= 1000

Wiki_Dict= {}
Doc_Dict= {}
Secondary_Index= {}
Index_Dict= {}

inverted_FilePath= "./inverted_index/index{}"
invertedIndex_FilePath= "./inverted_index/FinalIndex{}"
secondaryIndex_FilePath= "./SecondaryIndex"

total_tokens =0
stored_tokens =0

In [480]:
def write_secondaryIndex():
    secondaryIndex_file= open(secondaryIndex_FilePath, "wb")
    sorted_SecondaryIndex= dict(sorted(Secondary_Index.items(), key=lambda t: t[0]))
    pickle.dump(sorted_SecondaryIndex, secondaryIndex_file)  
    secondaryIndex_file.close()

In [481]:
def write_FinalIndex():
    global final_indexNum, stored_tokens
    sorted_IndexDict= dict(sorted(Index_Dict.items(), key=lambda t: t[0]))
    Secondary_Index[next(iter(sorted_IndexDict))]= str(final_indexNum)

    output_file= open(invertedIndex_FilePath.format(str(final_indexNum)), "w")
    for key, value in sorted_IndexDict.items():
        output_file.write(key+ ":"+ value+ '\n')
#             pickle.dump(sorted_IndexDict, output_file) 

    stored_tokens += len(Index_Dict)
    Index_Dict.clear()
    final_indexNum +=1
    output_file.close()
            

In [482]:
def evaluate_tfidf(word):
    posting_lists= Index_Dict[word].split('|')
    numOfDocs= len(posting_lists)
    global docId
    totalDocs= docId
    for pl in range(0, numOfDocs):
        numOfOccurences= int(posting_lists[pl].split('x')[1])
#         print (numOfOccurences)
        tfidf= math.log10(1+numOfOccurences) * math.log10(totalDocs/numOfDocs)
        posting_lists[pl]= posting_lists[pl].split('x')[0]+ 'x'+ str(tfidf)
        
    Index_Dict[word]= '|'.join(posting_lists)

In [483]:
def kWayMerge():
    global current_indexNum
    numOfFiles= current_indexNum
    input_files= []
    
    for file_num in range (1,numOfFiles):
        input_files.append(open(inverted_FilePath.format(str(file_num)), "r"))
    
    heap = [] 
    heapify(heap)  
    
    for file_num in range (1, current_indexNum):
        try:
            line= input_files[file_num-1].readline()
            word= line.split(":")[0]
            value= line.split(":")[1].split("\n")[0]
            heappush(heap, (word, value, file_num))
        except:
            print ("No lines is file {}".format(file_num))
            os.remove(inverted_FilePath.format(str(file_num)))
    
    while heap:
        entry= heappop(heap)
        word, line, file_num = entry[0], entry[1], entry[2]
        
        
        while heap and heap[0][0] == word:
            line += "|"+ heap[0][1]
            try:
                new_line= input_files[heap[0][2]-1].readline()
                new_word= new_line.split(":")[0]
                new_value= new_line.split(":")[1].split("\n")[0]
                heappush(heap, (new_word, new_value, heap[0][2]))
            except:
                print ("No lines is file {}".format(heap[0][2]))
#                 os.remove(inverted_FilePath.format(str(heap[0][2])))
            heappop(heap)
        
        Index_Dict[word]= line
        evaluate_tfidf(word)
            
        try:
            next_line= input_files[file_num-1].readline()
            next_word= next_line.split(":")[0]
            next_value= next_line.split(":")[1].split("\n")[0]
            heappush(heap, (next_word, next_value, file_num))
        except:
            print ("No lines is file {}".format(file_num))
            os.remove(inverted_FilePath.format(str(file_num)))
        
        if(len(Index_Dict) == max_IndexSize or len(heap) ==0):
            write_FinalIndex

In [484]:
def create_invertedIndex():
    global Wiki_Dict, current_indexNum
    sorted_WikiDict= sorted(Wiki_Dict.items(), key=lambda t: t[0])
    f = open(inverted_FilePath.format(str(current_indexNum)), "w")
    for key, value in sorted_WikiDict:
        line= key +':'+ value +"\n"
        f.write(line)
    f.close()
    current_indexNum +=1
    Wiki_Dict.clear()

In [485]:
class Page:
    def ProcessPage(self, pageID, content, index):
        for word in content:
            if word not in Doc_Dict:
                value= np.zeros(6, dtype= int)
                Doc_Dict[word]= value
                Doc_Dict[word][index] =1
            else:
                Doc_Dict[word][index] +=1

        if(index ==5):
            for key, value in Doc_Dict.items():
                total_frequency =0
                if key not in Wiki_Dict:                # pageID_totalcount:t1i2c3b4r5l6|pageID_totalcount:t1i2c3b4r5l6
                    Wiki_Dict[key]= str(pageID)+ '-'
                    for f in range (0, 6):
                        if(value[f] >0):
                            Wiki_Dict[key] += fields[f] + str(value[f])
                            total_frequency +=value[f]
                    Wiki_Dict[key] += 'x'+str(total_frequency)
                else:
                    Wiki_Dict[key] += '|'+ str(pageID)+ '-'
                    for f in range (0, 6):
                        if(value[f] >0):
                            Wiki_Dict[key] += fields[f] + str(value[f])
                            total_frequency +=value[f]
                    Wiki_Dict[key] += 'x'+str(total_frequency)
            Doc_Dict.clear()   

    def extract_pageDetails(self, text):
        global total_tokens
        lines= text.split('\n')
        line =0
        body_flag= False
        infoBox, category, body, references, links= [], [], [], [], []

        while line <len(lines):
            if any(element in lines[line] for element in UNWANTED_LINES) ==True:
                total_tokens += len(lines[line].split())
                line +=1
                
            elif "{{infobox" in lines[line]:
                openBrackets, closedBrackets= 1,0
                line +=1
                while line< len(lines):
                    if "{{" in lines[line]:
                        openBrackets +=1
                    if "}}" in lines[line]:
                        closedBrackets +=1

                    infoBox.extend(lines[line].split())
                    if(openBrackets == closedBrackets):
                        break
                    line +=1
                    
            elif "[[category:" in lines[line]:
                body_flag= True
                try:
                    category.extend((lines[line].split(':')[1].split(']]')[0]).split())
                except:
                    category.extend((lines[line].split(':')[1]).split())

            elif "==external links==" in lines[line] or "== external links ==" in lines[line]:
                body_flag= True
                line +=1
                while line< len(lines):
                    if "*[" in lines[line] or "* [" in lines[line]:
                        link= ""
                        while line< len(lines) and "]" not in lines[line]:
                            link += lines[line]
                            line+=1
                        if(line<len(lines)):
                            link += lines[line]
                        link = link.split('[')
                        if(len(link)> 1):
                            link= link[1].split(']')[0]
                            links.extend(link.split())
                    elif "[[category:" in lines[line]:
                        try:
                            category.extend((lines[line].split(':')[1].split(']]')[0]).split())
                        except:
                            category.extend((lines[line].split(':')[1]).split())
                    line+=1


            elif "==references==" in lines[line] or "== references ==" in lines[line]:
                body_flag= True
                line +=1
                while line< len(lines):
                    if "==" in lines[line] or "[[category:" in lines[line]:
                        line -=1
                        break
                    elif "{{cite" in lines[line] or "{{vcite" in lines[line]:
                        cite_title= lines[line].split("title=")
                        if(len(cite_title) >1):
                            cite_title= cite_title[1].split("|")[0]
                            references.extend(cite_title.split())
                    elif "{{" in lines[line] and "ref" not in lines[line]:
                        references.extend((lines[line].split("{{")[1].split("}}")[0]).split())
                    line+=1

            elif body_flag== False:
                body.extend(lines[line].split())
            line +=1  

        total_tokens+= len(infoBox)+len(category)+len(body)+len(references)+len(links)

        return ' '.join(infoBox), ' '.join(category), ' '.join(body), ' '.join(references), ' '.join(links)

In [486]:
class Data:
    def __init__(self):
        self.docId = 0
        self.title=""
        self.text = ""
        self.infoBox= ""
        self.category= ""
        self.body= ""
        self.references= ""
        self.links= ""
        self.stemmer= Stemmer.Stemmer("english")

    def set_data(self, docId, title, text):
        self.docId = docId
        self.title= title
        self.text = text
        
        page =Page()
        self.infoBox, self.category, self.body, self.references, self.links = page.extract_pageDetails(self.text)
        
        global total_tokens
        total_tokens+= len(self.title.split())
        
        self.title, self.infoBox, self.category, self.body, self.references, self.links= \
        self.cleanText(self.title), self.cleanText(self.infoBox), self.cleanText(self.category), self.cleanText(self.body), self.cleanText(self.references), self.cleanText(self.links)        
                
        page.ProcessPage(self.docId, self.title, 0)
        page.ProcessPage(self.docId, self.infoBox, 1)
        page.ProcessPage(self.docId, self.category, 2)
        page.ProcessPage(self.docId, self.body, 3)
        page.ProcessPage(self.docId, self.references, 4)
        page.ProcessPage(self.docId, self.links, 5)
        
    def isEnglish(self, s):
        try:
            s.encode(encoding='utf-8').decode('ascii')
        except UnicodeDecodeError:
            return False
        else:
            return True

    def cleanText(self, text):
        text = re.sub(r'<(.*?)>','',text) #Remove tags if any
        text = re.sub(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', '', text, flags=re.MULTILINE) #Remove Url
        text = re.sub(r'{\|(.*?)\|}', '', text, flags=re.MULTILINE) #Remove CSS
        text = re.sub(r'\[\[file:(.*?)\]\]', '', text, flags=re.MULTILINE) #Remove File
        text = re.sub(r'[^\w\s]' , '', text) #Remove Punctuations & Special Characters
        text = text.split()
        text = [x for x in text if x not in STOPWORDS and x not in URL_STOP_WORDS and (x.isdigit() and (len(x)<=2 or len(x)>=5)) ==False and bool(re.match('^(?=.*[a-zA-Z])(?=.*[0-9])', x)) ==False and self.isEnglish(x) and len(x)<=45] 
        text = [self.stemmer.stemWord(word) for word in text]
        return text

In [487]:
class WikiHandler( xml.sax.ContentHandler ):
    def __init__(self):
        self.CurrentData = ""
        self.title = ""
        self.text = ""

    # Call when an element starts
    def startElement(self, tag, attributes):
        self.CurrentData = tag
#         global docId
#         if tag == "page":
#             print (docId, end= " ")

    # Call when an elements ends
    def endElement(self, tag):
        global docId
        if self.CurrentData == "text":
            data= Data()
            data.set_data(docId, self.title, self.text)
            docId +=1
        self.text= ""
        self.CurrentData = ""

    # Call when a character is read
    def characters(self, content):
        if self.CurrentData == "title":
            self.title = content.lower()
        elif self.CurrentData == "text":
            self.text += content.lower()


In [488]:
if ( __name__ == "__main__"):
    start = datetime.datetime.now()
    
    # create an XMLReader
    parser = xml.sax.make_parser()
    # turn off namepsaces
    parser.setFeature(xml.sax.handler.feature_namespaces, 0)

    # override the default ContextHandler
    Handler = WikiHandler()
    parser.setContentHandler(Handler)
    
    files = glob.glob('../Phase2/temp_data/*.xml') 
    for file in files:
        start_file= datetime.datetime.now()
        parser.parse(file)
        create_invertedIndex()
        end_file = datetime.datetime.now()
        secs_file  = (end_file-start_file).seconds
        hr_file = int(secs_file/(60*60))
        rm_file = int(secs_file%(60*60))
        mn_file = int(rm_file/60)
        rm_file=int(rm_file%60)
        secs_file = int(rm_file)
        print()
        print ("--------------------done file{} in {} hrs {} mins {} secs--------------".format(str(current_indexNum-1), str(hr_file), str(mn_file), str(secs_file)))
        print ("{} documents were processed".format(docId))
    
    kWayMerge()
    write_secondaryIndex()
    
    end = datetime.datetime.now()
    secs  = (end-start).seconds
    hr = int(secs/(60*60))
    rm = int(secs%(60*60))
    mn = int(rm/60)
    rm=int(rm%60)
    secs = int(rm)
    
    print ("\nTotal Tokens= ", total_tokens)
    print ("\nStored Tokens= ", stored_tokens)
    print("\nIndexing Time : ",hr," hrs ",mn," mns",secs," secs")



--------------------done file1 in 0 hrs 0 mins 0 secs--------------
19 documents were processed

--------------------done file2 in 0 hrs 0 mins 0 secs--------------
66 documents were processed

--------------------done file3 in 0 hrs 0 mins 0 secs--------------
73 documents were processed

--------------------done file4 in 0 hrs 0 mins 0 secs--------------
98 documents were processed

--------------------done file5 in 0 hrs 0 mins 0 secs--------------
117 documents were processed
000  :  2-b1x1|65-b1x1
004  :  115-b1x1
0041  :  58-b1x1
0061  :  58-b1x1
009  :  115-b2x2
011  :  94-b1x1
012  :  115-b1x1
014  :  115-b1x1
015  :  115-b4x4
018  :  115-b2x2
0245  :  73-b1x1
025  :  115-b1x1
026  :  7-i1x1
0275  :  73-b1x1
031  :  85-b1x1|94-b1x1
0315  :  73-b1x1
032  :  73-b1x1
0335  :  73-b1x1
036  :  73-b1x1
0375  :  73-b1x1
0400  :  65-i1x1
0407  :  115-b1x1
045  :  73-b1x1
049  :  16-i1x1
050  :  115-b1x1
0500  :  65-i2x2
053  :  85-b1x1
055  :  115-b1x1
0600  :  65-i1x1
070  :  115-b1x

accept  :  1-b2x2|4-b3x3|6-b8x8|13-b3x3|14-b1x1|16-b3x3|67-b1x1|69-b5x5|70-b1x1|73-b6x6|78-b2x2|86-b1x1|88-b1x1|90-b1x1|94-b2x2|99-b4x4|109-b1x1
acceptedcit  :  109-b1x1
access  :  1-b2x2|7-b2x2|13-b1x1|14-b1x1|16-b4x4|18-b1x1|65-b6x6|67-b3x3|69-b1x1|85-b5x5|88-b1x1|90-b1x1
accessd  :  109-i1b11x12|115-b2x2|6-b1x1|7-b1x1|16-b1x1|65-b8x8|67-b1x1|69-b1x1|73-b2x2|77-b1x1|85-b3x3
accessdateapril  :  2-b1x1|10-i1x1|16-b2x2|65-b2x2|71-b3x3|73-b17x17|86-b1x1
accessdateaugust  :  2-b2x2|14-b12x12|65-b9x9|73-b3x3
accessdatedecemb  :  14-b5x5|16-b3x3|65-i1b5x6|69-i1b1x2|71-b2x2|73-b1x1
accessdatefebruari  :  2-b3x3|16-b2x2|65-b6x6|69-b6x6|73-b19x19
accessdatejanuari  :  2-b19x19|14-b1x1|16-b4x4|65-b2x2|73-b5x5
accessdatejuli  :  2-i1b1x2|14-b2x2|16-b1x1|65-i1b18x19|73-b1x1|94-b3x3
accessdatejun  :  2-b10x10|16-b16x16|65-b15x15|72-b1x1|73-b1x1|97-b2x2
accessdatemarch  :  2-b3x3|14-b6x6|16-b1x1|69-b2x2|73-b23x23
accessdatemay  :  2-b12x12|16-b14x14|65-b16x16|69-b2x2|73-b3x3
accessdatenovemb  :  2-

ancientgreeceorg  :  1-b1x1
ancillari  :  73-b1x1
ancsa  :  16-b2x2
andaman  :  4-b1x1
andbankbanc  :  7-b1x1
andcount  :  16-b1x1
andersen  :  72-b2x2
anderson  :  1-b1x1|14-b1x1|16-b1x1|69-b2x2|72-b21x21|73-b1x1|91-b1x1
andersoni  :  14-b1x1
andexcept  :  4-b1x1
andi  :  2-b2x2|13-b1x1|72-b1x1
andironandiron  :  65-b1x1
andor  :  109-b2x2|73-b1x1|88-b1x1|90-b1x1|94-b2x2
andorr  :  7-i1b1x2
andorra  :  55-t1x1|7-t1i12c1b212l5x231|8-b1x1
andorraandorra  :  7-b1x1
andorracit  :  7-b7x7
andorraconsel  :  7-b1x1
andorraconstitut  :  7-i1b1x2
andorraconstitutioncit  :  7-b1x1
andorracoprinc  :  7-i1b2x3
andorraeuropean  :  7-b1x1
andorragener  :  7-i1b1x2
andorrageolog  :  7-b1x1
andorrahead  :  7-b1x1
andorrajpgvolta  :  7-b1x1
andorrajudici  :  7-b1x1
andorrala  :  7-b2x2
andorralaw  :  7-b1x1
andorraloc  :  7-b1x1
andorramuslim  :  7-b1x1
andorran  :  7-i2b58x60
andorrana  :  7-b4x4
andorrano  :  7-b2x2
andorransandorran  :  7-i1x1
andorranslanguagecatalan  :  7-b1x1
andorranspanish  : 

bacteria  :  14-b2x2
bacterium  :  18-b1x1
bad  :  2-b1x1|13-b6x6|69-b2x2|70-b2x2|71-b1x1|82-b1x1|86-b1x1
badcock  :  109-b1x1
bade  :  1-b1x1
badi  :  85-b3x3
badisben  :  85-b1x1
baffl  :  97-b1x1
bafta  :  72-b1x1|97-b1x1
bag  :  2-b1x1|65-b1x1|73-b10x10|80-b1x1
bagdad  :  72-b2x2
bagel  :  2-b1x1
baghdati  :  2-b2x2
baghrir  :  85-b1x1
bahadur  :  78-b1x1
bahnar  :  4-i1b8x9
bain  :  94-b2x2
bainmari  :  90-b1x1
baisakhi  :  16-b1x1
bait  :  14-b1x1|73-b3x3
bakati  :  4-b1x1
bakatiq  :  4-b1x1
baker  :  109-b1x1|65-b2x2|67-b1x1|69-i1b1x2|72-b2x2|91-b1x1
bakermanskranenburg  :  109-b1x1
bakewel  :  89-b2x2
bakken  :  16-b2x2
baklawa  :  85-b1x1
bakshi  :  97-b1x1
bakunin  :  99-b3x3
bakuninist  :  99-b1x1
bal  :  109-b1x1
baladiyah  :  85-b1x1
balanc  :  1-b3x3|7-b1x1|9-b1x1|14-b2x2|18-b1x1|115-b1r4x5|69-b2x2|85-b1x1|90-b1x1
balancebal  :  69-b1x1
bald  :  16-b1x1
baldung  :  70-b2x2
baldwin  :  65-i1b3x4
balines  :  88-b1x1
ball  :  115-b1x1|2-b2x2|70-b4x4|73-b2x2|91-b1x1|97-b1x1
b

buchanan  :  69-i1b4x5
bucharest  :  67-b1x1
buckley  :  80-b2x2
buckwalterpoza  :  65-b1x1
budapest  :  1-b1x1|72-b1x1
budd  :  2-b3x3
buddhism  :  65-b1x1|78-b4x4
buddhist  :  90-b1x1
buddw  :  72-b1x1
budg  :  90-b1x1
budgebudg  :  2-b1x1
budget  :  115-b1x1|65-b3x3|7-b6x6|16-b1x1|75-i1x1|85-b7x7
budgetari  :  85-b2x2
budgethes  :  2-b1x1
buel  :  18-b1x1|89-b1x1
buell  :  69-b2x2
buena  :  75-i1x1
buff  :  73-b1x1
buffalo  :  10-i1x1|18-b1x1
buffer  :  13-b1x1|18-b2x2|78-b1x1
bufo  :  14-b7x7
bufonida  :  14-b1x1
bufotoxin  :  14-b1x1
bug  :  97-b4x4
bugsi  :  72-b1x1
build  :  1-b9x9|2-b5x5|7-b2x2|13-b2x2|14-b1x1|16-b2x2|18-b1x1|65-b10x10|67-b1x1|69-b1x1|70-b1x1|78-b5x5|80-b1x1|82-b1x1|85-b1x1|86-b1x1|88-b1x1|92-b1x1|94-b1x1|96-b2x2|97-b1x1|99-b2x2|115-b1x1
builder  :  1-b1x1|7-b1x1|86-b1x1
buildingintegr  :  18-b1x1
built  :  1-b23x23|7-b3x3|13-b1x1|14-b3x3|16-b3x3|65-b5x5|67-b3x3|85-b3x3|86-b1x1|88-b1x1|90-b1x1|99-b2x2
builthillison  :  89-b1x1
buitelaar  :  109-b1x1
bukharin  :

...

selfish  :  78-b5x5|80-b2x2
selfishnessbook  :  80-b1x1
selfless  :  78-b5x5
selflesscit  :  78-b1x1
selfmad  :  69-b1x1|78-b1x1|86-b1x1
selfmanag  :  99-b3x3
selfmanagementsfnvan  :  99-b1x1
selfmasteri  :  99-b1x1
selfperpetu  :  99-b1x1
selfpollin  :  18-b1x1
selfproclaim  :  7-b1x1
selfreport  :  78-b1x1
selfreportedcit  :  65-b1x1
selfsacrific  :  78-b1x1
selfsacrifici  :  78-b1x1
selftitl  :  91-b1x1
selftransform  :  90-b1x1
selinus  :  1-b1x1
selinustempl  :  1-b1x1
sell  :  1-b1x1|13-b2x2|16-b1x1|67-b1x1|73-b2x2|97-b1x1
sellcit  :  16-b1x1
seller  :  2-b2x2
sellout  :  2-b1x1|86-b1x1
selma  :  65-b4x4
selon  :  85-b1x1
seltzer  :  109-b2x2
semi  :  85-b1x1
semiarid  :  18-b1x1|85-b1x1
semicurs  :  58-b1x1
semiempir  :  115-b1x1
semifin  :  2-b16x16
semifluid  :  70-b1x1
semin  :  88-b1x1|90-b1x1
semiot  :  88-b1x1
semipresidenti  :  85-i1b1x2
semiprofession  :  16-b1x1|65-b1x1
semistructur  :  109-b1x1
semit  :  58-b1x1|6-i1b43l2x46|85-b1x1
semiterrestri  :  14-b1x1
semiticcar

steadman  :  13-b1x1
steal  :  1-b2x2|18-b1x1
steam  :  86-b1x1
steamboat  :  97-b1x1
steamboatriv  :  69-b1x1
stebbin  :  14-b24x24
steel  :  65-b9x9|67-b1x1|69-b1x1|85-b2x2|86-b6x6
steelcit  :  65-b1x1
steeler  :  10-i4x4
steelmak  :  65-b1x1
steelstainless  :  65-b1x1
steep  :  115-b1x1|16-b1x1
stefan  :  2-b2x2
stefanato  :  109-b1x1
steffi  :  2-c1b10l1x12
steinberg  :  78-b1x1
steinbergfirst  :  78-b1x1
steiner  :  90-b1x1
steinfeld  :  18-b1x1
steinhausen  :  109-b1x1
stela  :  67-b1x1
stellar  :  115-b1x1|92-b1x1
stem  :  14-b3x3|18-b2x2|67-b1x1|90-b1x1|97-b2x2
step  :  1-b1x1|2-b1x1|7-b1x1|14-b1x1|109-b1x1|65-b1x1|72-b1x1|81-b1x1|85-b2x2|90-b2x2
stephan  :  109-b1x1
stephani  :  2-b1x1|18-b1x1
stephanus  :  67-b1x1
stephen  :  1-b1x1|115-b1x1|65-b5x5|69-b5x5|70-b2x2|72-b14x14|85-b1x1
stepheni  :  72-b5x5
stephens  :  7-b1x1
stephensstephen  :  69-b1x1
stepmoth  :  69-b2x2
stepmothertibullus  :  1-b1x1
stepp  :  85-b1x1
stereo  :  71-b1x1
stereospondylistereospondyl  :  14-b1x1

tinseltown  :  82-b1x1
tintin  :  97-b2x2
tip  :  2-b1x1|14-b2x2|16-b1x1|78-b1x1
tipasa  :  85-b1x1
tippett  :  97-b1x1
tire  :  1-b2x2|14-b1x1|16-b1x1|65-b2x2
tired  :  109-b1x1
tiresom  :  70-b1x1
tirthankara  :  78-b1x1
tissu  :  109-b1x1|14-b3x3|89-b1x1
tit  :  78-b2x2
titan  :  1-b5x5|10-i4x4|67-b3x3|72-b2x2|73-b2x2
titanessasteria  :  1-b2x2
titanessdion  :  1-b1x1
titanomachi  :  1-b2x2
titl  :  1-b7x7|2-b38x38|4-b1x1|6-b12x12|7-i2b7x9|13-b3x3|14-b6x6|16-b4x4|65-b8x8|67-b5x5|69-b6x6|70-b2x2|71-b1x1|73-i1b8x9|76-i1x1|77-b1x1|78-b15r1x16|80-b1x1|81-b1x1|85-b10x10|86-b1x1|88-b4x4|90-b5x5|91-b1x1|99-b1x1|109-i2b134x136|115-b8x8
title_origanim  :  13-i1x1
titlea  :  109-b1x1|2-b1x1|14-b4x4|18-b3x3|65-i1b1x2|78-b2x2|85-b3x3|94-b1x1
titleabc  :  73-b2x2
titleabiot  :  14-b1x1
titleabout  :  65-b1x1|7-b1x1|16-b1x1|92-b2x2
titleabraham  :  69-i1b1x2
titleac  :  65-b1x1
titleacademi  :  72-b1x1|73-b11x11
titleactrius  :  75-b1x1
titleadam  :  73-b1x1
titleadapt  :  14-b2x2
titleafrica  : 

unfertilis  :  14-b2x2
unfilt  :  18-b1x1
unfinish  :  67-b1x1
unfit  :  18-b1x1
unflatt  :  13-b1x1
unfold  :  88-b1x1
unforgiven  :  72-b1x1
unforgotten  :  85-b1x1
unfortun  :  72-b1x1
ungener  :  69-b1x1
unhcr  :  85-b2x2
unhealthi  :  78-b2x2
unhind  :  67-b1x1
unicellular  :  14-b1x1
unicod  :  94-i2b13x15
unidentifi  :  109-b1x1|91-b1x1
unif  :  7-b1x1|86-b3x3
unifi  :  16-b1x1|85-b1x1|97-b1x1
uniform  :  65-b1x1|7-b2x2|16-b1x1|18-b1x1|86-b1x1
uniformitarianismgeolog  :  70-b1x1
unimagin  :  75-b1x1
unimak  :  16-b1x1
unimport  :  78-b1x1
unimpress  :  80-b1x1
unincorpor  :  85-b1x1
uninhabit  :  16-b1x1|67-b1x1
uniniti  :  90-b1x1
unintellig  :  90-b1x1
unintend  :  78-b1x1
union  :  1-b1x1|7-b9x9|13-b8x8|14-b1x1|16-b2x2|18-b7x7|65-b4x4|69-i2c1b50x53|73-b1x1|80-i1b3x4|81-b2x2|85-c2b9x11|90-b1x1|92-b1l1x2|99-b3x3
unioncit  :  80-b1x1
unionf  :  13-b1x1
unionhitlerstalin  :  13-b1x1
unionist  :  69-b4x4
unionlabour  :  7-b1x1
unionlist  :  16-b1x1|65-b1x1
unionnbsp  :  69-b1x1
un

western  :  1-b1x1|2-b1x1|4-b3x3|6-b4x4|13-b3x3|14-b2x2|16-c1b2x3|18-b1x1|58-b1x1|65-b3x3|69-b4x4|70-i1c2b5x8|72-b2x2|82-c1b1x2|85-b5x5|88-b4x4|90-b13l1x14|96-b1x1
westernmost  :  16-b1x1
westeuropean  :  97-b1x1
westjack  :  88-b1x1
westman  :  85-b1x1
westview  :  88-b1x1
westward  :  4-b1x1
westwestcit  :  10-i1x1
wet  :  14-b3x3|67-b1x1|70-b2x2|86-b1x1
wetdri  :  65-b1x1
wetherbi  :  109-b1x1
wetland  :  16-b1x1|18-b2x2
wetnwild  :  2-b1x1
wetumpka  :  65-b6x6
weyl  :  72-b2x2
wfiq  :  65-b1x1
wfna  :  65-b1x1
wgiq  :  65-b1x1
whale  :  16-i1b2x3
what  :  109-b1x1|73-b1x1
whatev  :  1-b2x2|13-b2x2|16-b1x1|70-b3x3|97-b1x1
whdf  :  65-b1x1
wheat  :  18-b8x8|86-b1x1
wheaton  :  2-b1x1
wheatrel  :  109-b1x1
wheelcit  :  16-b1x1
wheeler  :  14-b1x1|72-b28x28
whenc  :  67-b1x1
whenev  :  13-b1x1|86-b1x1
wherea  :  1-b1x1|4-b1x1|6-b1x1|14-b1x1|16-b1x1|115-b6x6|58-b1x1|67-b1x1|70-b6x6|78-b1x1
wherebi  :  6-b1x1|7-b1x1|18-b1x1|90-b1x1
wherein  :  70-b1x1
wherev  :  1-b1x1
whet  :  67-b1x1
w

In [489]:
# f= open("./inverted_index/FinalIndex1", "rb")
# db= pickle.load(f)
# db

In [490]:
# f= open("./SecondaryIndex", "rb")
# db= pickle.load(f)
# db